# "Informational Theoretical Neural Network Pruning"
> "This article discusses pruning and its implications to neural network design, in particular network streamlining. And this is done via information theoretical tools which inform us which layers, kernels, parameters are most informative and need to be preserved at all costs and which we can dispense with all the while incurring negligeable information loss. "

- toc: true
- branch: master
- bibliography: true
- math: true
- badges: true
- comments: true
- categories: [Prospective Research, Information Bottleneck, Information Plane, Convolutional Neural Networks, Tiny Machine Learning]
- image: /images/camacho.png
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

![](/images/camacho.png "Fig 1. Obtained from Cezanne Camacho's blog <cezannec.github.io/Convolutional_Neural_Networks/>, which contains in-depth coverage of CNNs")

## Introduction

One way to measure the efficiency of convolutional neural networks (CNNs) is through compression. A practical goal is to design such networks with the least possible amount of parameters and layers for a given target accuracy using Information Theory . 

A paradigm known as Tiny ML [Stewart, 2020] could be easily enhanced with information theoretical pruning, which takes inspiration from the Information Bottleneck problem [Tishby et al., 2000; Tishby et al., 2015]. In the remainder of this text, I will discuss some major concepts in hyperparameter pruning. The main idea is to find methods that define the number of layers and filters most adequate for a given target accuracy and processing speed.

## Convolutional Neural Networks
Convolutional networks are powerful in learning features from a large set of input variables and in categorising and predicting otherwise extremely difficult output variables. In particular, these networks are very well suited for unstructured data, especially Computer Vision. Images are complex to process since they usually contain a large amount of input variables (pixels). In addition, the desired output is usually related to the input via complex non-linear functions, most of which cannot even be expressed in analytic form. 

Convolutional neural networks use kernels to filter specific features important to the output. This is best understood in terms of outlines within images. Kernels may be designed to detect lines, curves, but also very abstract and complex features [Goodfellow et al., 2016]. In fact, kernels do not even need to be designed; they can be learned [Goodfellow et al., 2016]. This property makes CNNs extremely powerful. Once the network goes some layers deep, after hidden layers such as convolutional kernels, fully-connected (FC) layers, linear rectifier units (ReLUs), and batch normalization, the system’s inner variables represent information that cannot be intuitively described or mathematically formulated [Brownlee, 2019]. It is this level of abstraction that allows CNNs to solve complex problems. Convolutional architectures such as ResNets [He et al., 2015], AlexNets [Krizhevsky et al., 2012] and VGG [Simonyan et al., 2014] predict outcomes with high accuracy essentially because they are able to learn optimizing parameters coming from many parallel filters which extract features through many different paths [Nielsen, 2015]. Kernels in parallel identify different image characteristics, increasing the network’s strength in learning from the data.

The staggering number of parameters of CNNs can cause the model to overfit the training data and, as a result, generalize poorly to the test set. Common approaches to mitigate this problem are regularization and pooling. Adding regularization terms to the cost function penalizes the overuse of parameters. Pooling summarizes the individual outputs of adjacent filters usually by subsuming them to a mean or maximum output value [Goodfellow et al., 2016]. By summarizing and averaging data, parameter cardinality is reduced and randomness is smoothed over. However, the best approach to improve generalization is still parameter pruning [Li et al., 2016; Blalock et al., 2020], which means removing neurons or doing away with entire filters (see Fig. 1). With fewer parameters, CNN-produced models generalize much more reliably. 

![](/images/kernel_pruning.png "Fig 2. Pruning a filter in layer i results in removing its feature map and kernels in layer i+1. Source: [Li et al., 2016]")

Determining which connections, weights and biases are essential requires a prior understanding of how input variables relate to each other and to the output. That could be achieved by calculating the mutual information between variables. The drawback is that this requires the estimation of multivariate joint probability distributions. The complex and rich relationship between variables within a network makes this an intractable problem. Besides, the deep layers pose another problem. Since they contain very abstract information, we do not even know what their inner variables mean and how they relate to the input and to the output. To perform standard statistical analysis on them is impractical. 

## Tiny Machine Learning
More than addressing overfitting, there are other advantages to pruning, namely, energy-efficient computing and a lower memory storage footprint [Stewart, 2020]. Fewer parameters mean less computation in the feedforward propagation. In addition, the amount of memory necessary to store the network’s parameters is significantly reduced. This is well evidenced in the framework of Tiny ML [Stewart, 2020]. In embedded systems, the limitation of the processing units forces Machine Learning architectures to be as lean as possible. This is usually achieved via a mix of pruning, weight quantization, and compression [Han et al., 2016]. Quantization involves forcing the weights to belong to a predefined code set. This limits the number of bits used. And since the probability distribution of the weights is non-uniform, Huffman coding can be used to reduce even further the number of stored bits. In deep compression [Han et al., 2016], the usual pruning method discards neurons attached with weights close to zero. However, this approach does not take into account dependencies at deeper levels. Seemingly unimportant connections at a lower level may have a substantial influence on top layers.  

Tiny ML is an interesting paradigm for compressing a neural network. Many of its design criteria can serve as guidelines for efficient CNN design. Yet, its ideas can be refined. An interesting extra step could consist in the use of autoencoders. It is interesting to note that autoencoders are closely related to the communication problem in that they estimate the output most similar to the input given the network's constraints. In fact, the number of nodes in the hidden layers is less than in the input set. This is done to extract only the essential features of the data [Yu et al., 2019]. Autoencoders work as a form of lossy compression, or, still, as a form of dimensionality reduction. This is similar to what PCA does, but with the caveat that autoencoders encode non-linearities [Yu et al., 2019]. Autoencoders could be a front-end module to CNNs, extracting from the input data only the most relevant features. 

## Information Bottleneck Applied to Pruning

The next step is making Tiny ML more powerful, and this involves changing the pruning criteria. Research suggests that mutual information is a much more robust design rule than the absolute value of the parameters [Yu, 2020]. A neural network can be effectively described by its Information Bottleneck (IB): the mutual information between the inner variables and the network's input and between the inner variables and the output [Tishby et al., 2000; Tishby et al., 2015; Yu et al., 2020]. According to [Tishby et al., 2000], the IB is given by

\\[
\begin{equation}
L[p_{X \mid T}(x \mid t)] = I(X; T)−\beta I(T; Y), 
\end{equation}
\\]

where we see that X, T, Y form a Markov Chain (we will let the reader verifiy that this is the case given the formula above), X and Y denote the input batch and the corresponding output, respectively, T denotes the filter's feature map, and \beta the Lagrange multiplier, which controls the trade-off between compression and information preservation [Tishby, 2015]. Assume that T is a quantized codebook, i.e., a compressed representation of X. It is Y’s minimum sufficient statistic, which means it captures the entire mutual information I(X; Y) (all the relevant data about Y). The optimization objective is to minimize I(X; T) to obtain the simplest (most compressed) statistic under the constraint of maximum I(T; Y) [Tishby et al., 2000; Tishby et al., 2015]. This implies that we want a filter that compresses the input data as much as possible while capturing the most relevant information about Y [Tishby et al., 2015]. 

Intuitively, a small value of the objective function indicates that the filter feature map T results in a compressed representation of X that is relevant to Y. Therefore, the smaller the objective value, the higher the importance of T. This information is key to determining whether a filter should or not be preserved in the final structure.

It is interesting to note that the IB method is intrinsically related to Rate Distortion Theory [Tishby et al., 2020], which is another interesting connection to the lossy compression problem in Communications. The Information Bottleneck is an instance of Rate Distortion where the distortion measure (the Kullback-Leibler divergence - this measure is very closely related to the cross-entropy) varies according to the filter feature map T that is being learned [Tishby et al., 2015]. As the information theoretical operators require very complex joint probability density functions, an alternative is to estimate the IB via a matrix functional directly computed from the data within the network [Yu et al., 2020]. This functional is inspired on Renyi’s $\alpha$-entropy: a measure of a variable's uncertainty similar to Shannon's traditional definition of entropy [Yu, 2020]. 

![](/images/pid.png "Fig 3. Synergy and redundancy among feature maps. Source: [Yu et al.,2020]")

Another practical way to approach the IB problem is Partial Information Decomposition (PID) [Wibral et al., 2017; Yu et al., 2020]. This method breaks down the multivariate mutual information into synergies and redundancies (see Fig. 2) [Yu et al., 2020]. Two filter feature maps $T_1$ and $T_2$ are synergetic when they rely on each other to explain the mutual information of the output. And they are redundant when they explain the same amount of information separately. 

There exists a trade-off between synergy and redundancy. The basic idea is to attempt to maximize synergies and minimize redundancies and retain the fewest possible variables [Yu et al., 2020]. The trade-offs can be better analysed with Information Plane (IP) curves. These curves indicate more precisely how information evolves within the network through its epochs and give hints on how the network should be designed.

On future posts, I will talk more about the IP curves and how they inform decisions on neural network design. I will also explain some of the information theoretical terminology that appeared in this brief.

## References

[1] Davis Blalock, Jose Javier Gonzalez Ortiz, Jonathan Frankle, and John Guttag. What is the State of Neural Network Pruning? arXiv:2003.03033 [cs, stat], March 2020. arXiv: 2003.03033.

[2] P. R. Branco da Silva and D. Silva. Low­-complexity multilevel ldpc lattices and a generalization of construction D’. In 2018 IEEE International Symposium on Information Theory (ISIT), pages 626–630, 2018.

[3] Jason Brownlee. How Do Convolutional Layers Work in Deep Learning Neural Networks?, April 2019.

[4] Thomas M. Cover and Joy A. Thomas. Elements of Information Theory (Wiley Series in Telecommunications and Signal Processing). Wiley­Interscience, USA, 2006.

[5] Ian Goodfellow, Yoshua Bengio, and Aaron Courville. Deep Learning. MIT Press, 2016. http://www.deeplearningbook.org.

[6] Bruna Gregory Palm, Dimas Irion Alves, Mats Pettersson, Viet Thuy Vu, R. Machado, R. Cintra, Fábio Bayer, Patrik Dammert, and Hans Hellsten. Wavelength­resolution sar ground scene prediction based on image stack. Sen­sors, 20:2008, 04 2020.

[7] Song Han, Huizi Mao, and William J. Dally. Deep Compression: Compressing Deep Neural Networks with Pruning, Trained Quantization and Huffman Coding. arXiv:1510.00149 [cs], February 2016. arXiv: 1510.00149.

[8] Kaiming He, Xiangyu Zhang, Shaoqing Ren, and Jian Sun. Deep Residual Learning for Image Recognition. December 2015.

[9] Alex Krizhevsky, Ilya Sutskever, and Geoffrey E Hinton. ImageNet Classification with Deep Convolutional Neural Networks. In F. Pereira, C. J. C. Burges, L. Bot­tou, and K. Q. Weinberger, editors, Advances in Neural Information Processing Systems 25, pages 1097–1105. Curran Associates, Inc., 2012.

[10] Hao Li,Asim Kadav,Igor Durdanovic,Hanan Samet,and Hans Peter Graf. Pruning Filters for Efficient ConvNets. November 2016.

[11] Michael A. Nielsen. Neural Networks and Deep Learning. 2015. Publisher: Determination Press.

[12] Matthew Stewart Researcher, PhD. Tiny Machine Learning: The Next AI Revolution, October 2020.

[13] Karen Simonyan and Andrew Zisserman. Very Deep Convolutional Networks for Large­Scale Image Recognition. September 2014.

[14] Naftali Tishby, Fernando C. Pereira, and William Bialek. The information bottle­neck method. arXiv:physics/0004057, April 2000. arXiv: physics/0004057.6

[15] Naftali Tishby and Noga Zaslavsky. Deep Learning and the Information Bottleneck Principle. arXiv:1503.02406 [cs], March 2015. arXiv: 1503.02406.

[16] Tianchen Wang, Jinjun Xiong, Xiaowei Xu, and Yiyu Shi. SCNN: A General Distribution based Statistical Convolutional Neural Network with Application to Video Object Detection. arXiv:1903.07663 [cs, stat], March 2019. arXiv: 1903.07663.

[17] Michael Wibral, Viola Priesemann, Jim W. Kay, Joseph T. Lizier, and William A. Phillips. Partial information decomposition as a unified approach to the specification of neural goal functions. Brain and Cognition, 112:25 – 38, 2017.

[18] Shujian Yu and Jose Principe. Understanding autoencoders with information the­oretic concepts. Neural Networks, 05 2019.

[19] Shujian Yu, Kristoffer Wickstrøm, Robert Jenssen, and Jose C. Principe. Understanding Convolutional Neural Networks with Information Theory: An Initial Explo­ration. arXiv:1804.06537 [cs, math, stat], January 2020. arXiv: 1804.06537